In [1]:
pip install wfdb

In [2]:
from tensorflow.keras.optimizers import Adam
import numpy as np
import keras.backend as K
import tensorflow as tf
import os
import pickle
from keras.models import model_from_json
import sys
from keras.layers import Convolution1D, MaxPooling1D, Reshape, PReLU, Input, Bidirectional, LSTM
from keras.layers import Dense, Dropout,MaxPooling2D, Activation, Flatten, Convolution2D, InputLayer, concatenate
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.callbacks import Callback
from tensorflow.keras.utils import to_categorical
import random
import h5py
import time
import wfdb
from scipy.signal import hilbert
from tensorflow.keras.utils import plot_model 
import requests

In [3]:
start_dir = '/content/'

# **Download Data**

In [4]:
!wget https://physionet.org/files/challenge-2018/1.0.0/RECORDS?download

--2021-11-14 08:00:46--  https://physionet.org/files/challenge-2018/1.0.0/RECORDS?download
Resolving physionet.org (physionet.org)... 18.18.42.54
Connecting to physionet.org (physionet.org)|18.18.42.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53551 (52K) [text/plain]
Saving to: ‘RECORDS?download’

RECORDS?download    100%[===================>]  52.30K  --.-KB/s    in 0.07s   

2021-11-14 08:00:47 (756 KB/s) - ‘RECORDS?download’ saved [53551/53551]



In [5]:
f = open("/content/RECORDS?download", 'r')
Lines = f.readlines()
os.mkdir("/content/data/")
os.mkdir("/content/data/test/")
os.mkdir("/content/data/training")

In [6]:
train_counter = 0
test_counter = 0
for l in Lines:
  if str(l).__contains__("train"):
    train_counter += 1
    if train_counter >= 10:
      break
    name = l.strip().split('/')[1]
    os.mkdir("/content/data/training/" + name + "/")
    # os.chdir("/content/data/test/" + name + "/")
    url = 'https://physionet.org/files/challenge-2018/1.0.0/'+ l.strip() + "-arousal.mat"
    r = requests.get(url, allow_redirects=True)
    open('/content/data/' + l.strip() + "-arousal.mat" , 'wb').write(r.content)

    url = 'https://physionet.org/files/challenge-2018/1.0.0/'+ l.strip() + ".arousal"
    r = requests.get(url, allow_redirects=True)
    open('/content/data/' + l.strip() + ".arousal" , 'wb').write(r.content)

    url = 'https://physionet.org/files/challenge-2018/1.0.0/'+ l.strip() + ".hea"
    r = requests.get(url, allow_redirects=True)
    open('/content/data/' + l.strip() + ".hea" , 'wb').write(r.content)

    url = 'https://physionet.org/files/challenge-2018/1.0.0/'+ l.strip() + ".mat"
    r = requests.get(url, allow_redirects=True)
    open('/content/data/' + l.strip() + ".mat" , 'wb').write(r.content)
  else:
    test_counter += 1
    if test_counter >= 2:
        continue
    name = l.strip().split('/')[1]
    if(os.path.isdir("/content/data/test/" + name + "/")):
      continue
    os.mkdir("/content/data/test/" + name + "/")
    # os.chdir("/content/data/test/" + name + "/")
    url = 'https://physionet.org/files/challenge-2018/1.0.0/'+ l.strip() + ".hea"
    r = requests.get(url, allow_redirects=True)
    open('/content/data/' + l.strip() + ".hea" , 'wb').write(r.content)

    url = 'https://physionet.org/files/challenge-2018/1.0.0/'+ l.strip() + ".mat"
    r = requests.get(url, allow_redirects=True)
    open('/content/data/' + l.strip() + ".mat" , 'wb').write(r.content)
  

# **Pre-Processing**

In [7]:
########################################################
#Script for preprocessing of raw data into time windows#
########################################################


#raw data directory
fileDir = start_dir + 'data/training/'
#parent of directory where to save windowed data
resDir = start_dir + 'data/'


fileList = os.listdir(fileDir)
fileList = [f for f in fileList if f[:2] == 'tr']



windowSize = 30
downSampling = 4 #downsampling ratio for the signal
fs = 200
numOfChannels = 13
win_size_fs = int(round(windowSize*(fs/float(downSampling)))) #1500
overlap = False
cumNeg = 0
cumPos = 0

# -------------------------------------- for all training data
for i in range(0,len(fileList)):
    start_time = time.time()
    print(str(i+1))

    currPatient = fileList[i]
    currSubFolder =  fileDir + currPatient + '/'
    dataFile = currSubFolder + currPatient
    val, _ = wfdb.rdsamp(dataFile)
    val = np.float16(val)
    arousalFile = currSubFolder + currPatient + '-arousal.mat'
    with h5py.File(arousalFile, 'r') as f:
        arousalLabel = f['data']['arousals']
        arousalLabel = arousalLabel[:]
         
        
    val = val[::downSampling,:]


    if overlap:
        templName = 'overlap_'
    else:
        templName = 'non_overlap_'
        
    templName = templName +  str(windowSize)
    
    userCacheDir = resDir + 'cache/'  
    if not os.path.isdir(userCacheDir):
        os.mkdir(userCacheDir)
    userCacheDir = userCacheDir + templName + '/'
    if not os.path.isdir(userCacheDir):
        os.mkdir(userCacheDir)
    
    userCacheDir = userCacheDir + fileList[i] + '/'
    if not os.path.isdir(userCacheDir):
        os.mkdir(userCacheDir)
        
    
    arousalLabel = arousalLabel[::downSampling]
    
    length = len(val)
    startIdx = 0
    finIdx = win_size_fs
    if overlap:
        deltaWindow = int(np.floor(win_size_fs/2.))
    else:
        deltaWindow = int(win_size_fs)
    
    winCount = 1
    x_temp = []
    y_temp = []
    # windowing for each one
    while finIdx <= length:
        winVal = val[startIdx:finIdx,:]
        currLabelVec = np.array(arousalLabel[startIdx:finIdx]);
        currLabelVoting = [np.sum(currLabelVec == -1),np.sum(currLabelVec == 0),
                           np.sum(currLabelVec == 1)]
        idx = np.argmax(currLabelVoting);
        idx = idx - 1;
        #consider max number of 0 -1 and 1 in each window(vote)
        y_temp.append(idx)
        x_temp.append(winVal)
        winCount = winCount + 1
        startIdx = startIdx + deltaWindow
        finIdx = finIdx + deltaWindow
    x_temp = np.array(x_temp,dtype = np.float16 )
    y_aux = []
    valid_count = 0;
    # consider number of arousal marked windows
    pos = np.sum(np.asarray(y_temp) == 1.)
    neg = np.sum(np.asarray(y_temp) == 0.)
    
    for j in range(np.size(x_temp,0)):
        if y_temp[j] >= 0: # do not consider -1 marked
            pickle.dump(x_temp[j],open(userCacheDir + 'x_' + str(valid_count) + '.p','wb+'))
            
            if j > 0:
                prevIdx = j - 1
            else:
                prevIdx = j
            if y_temp[prevIdx] == -1:
                pickle.dump(x_temp[prevIdx],open(userCacheDir + 'x_prev_' + str(valid_count) + '.p','wb+'))
                
            if j < (len(y_temp) - 1):
                nextIdx = j + 1
            else:
                nextIdx = j
            if y_temp[nextIdx] == -1:
                pickle.dump(x_temp[nextIdx],open(userCacheDir + 'x_next_' + str(valid_count) + '.p','wb+'))
            
            valid_count = valid_count + 1
            y_aux.append(y_temp[j])
  
    pickle.dump(np.asarray(y_aux,dtype = np.int16),open(userCacheDir + 'y.p','wb+'))
    print("--- %s seconds ---" % (time.time() - start_time))


1
--- 2.113645076751709 seconds ---
2
--- 1.5264689922332764 seconds ---
3
--- 2.2337214946746826 seconds ---
4
--- 1.6484529972076416 seconds ---
5
--- 1.8462467193603516 seconds ---
6
--- 1.8016185760498047 seconds ---
7
--- 1.945340633392334 seconds ---
8
--- 1.6989026069641113 seconds ---
9
--- 1.7636492252349854 seconds ---


# **Main part**
1.   Define the parameter 
2.   Define Model's parameter

In [8]:
############################################################
###CONFIGURATION STUFF, depend on specifc of machine   #####
############################################################
numOfCPU = 2
workers = 2
dataDir = start_dir + 'data/'
#setting GPU device to use.
os.environ['CUDA_VISIBLE_DEVICES']="1"

#setting maximum number of cores in tensorflow backend
config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=numOfCPU, inter_op_parallelism_threads=numOfCPU, \
                        allow_soft_placement=True, device_count = {'CPU': numOfCPU})
session = tf.compat.v1.Session(config=config)
K.set_session(session)

In [9]:
#####################################
###SETTING UP SOME PARAMETERS...#####
#####################################
training2do = True
validation2do = True
testing2do = True
totalNumOfUser = 7
siameseNet = 2
channels2Use = [[3],[6],[8],[10],[11],[12]]
convModelDims = [1,1,1,1,1,1]
typeOfModels = ['new_cnn2','new_cnn2','new_cnn2','new_cnn2','mlp','new_cnn2']
mergingMLP = 'a_bit_deep_wider'
# modelID = '3_6_8_10_11_12'
useBatchNorm = False
prelu = True
winSize = 30
modelID = "1"
data_path = start_dir + 'data/cache/non_overlap_' + str(winSize) + '/'
# data_path = dataDir + 'cache/non_overlap_' + str(winSize) + '/'
test_data_dir = start_dir + 'data/test/'
batch_size = 32
useGenerator = False
n_ep = 30
samplesPerWin = 1400
valNum = 1
testNum = 1

In [10]:
######################################################
###DEFINING TRAINING, VALIDATION AND TEST SETS...#####
######################################################
def get_num_of_samples(data_dir,userIDs):
    n_samples = 0
    n_pos = 0
    n_neg = 0
    for currUser in userIDs:
        print(data_dir + currUser + '/y.p')
        y = pickle.load(open(data_dir + currUser + '/y.p', 'rb'))
        n_samples = n_samples + len(y)
        n_pos = n_pos + np.sum(   np.asarray(y) == 1   )
        n_neg = n_neg + np.sum(   np.asarray(y) == 0   )
    
    return n_samples, n_pos, n_neg

users = os.listdir(data_path)
users = [u for u in users if u[:2] == 'tr']
users = users[:totalNumOfUser]
users.sort()

if testing2do:
    test_users = os.listdir(test_data_dir)
    test_users = [u for u in test_users if u[:2] == 'te']
    test_users.sort()

userIDs = users[(valNum+testNum):]
testUserIDs = users[:testNum]
valUserIDs = users[testNum:(testNum+valNum)]

n_samples, n_pos, n_neg = get_num_of_samples(data_path,userIDs)

print('num of samples: ' + str(n_samples))
print('Positive to Negative Ratio: ' + str( float(n_pos)/float(n_neg) ))

/content/data/cache/non_overlap_30/tr03-0052/y.p
/content/data/cache/non_overlap_30/tr03-0078/y.p
/content/data/cache/non_overlap_30/tr03-0079/y.p
/content/data/cache/non_overlap_30/tr03-0086/y.p
/content/data/cache/non_overlap_30/tr03-0087/y.p
num of samples: 3280
Positive to Negative Ratio: 0.028858218318695106


In [11]:
######################################################
###DEFINING Layes Parameter                      #####
######################################################
newConvFilters = [8   ,8    ,8   ,16  ,32  ,32]
newConvKernels = [16   ,16    ,16   ,32  ,32   ,64]
newConvPool = [False,True,False,True,False,True]
# channels2Use = [[3],[6],[7],[9],[11],[12]]
convFilters = []
convKernels = []
convPool = []
convDO = []

denseUnits = []
denseDO = []
rnnUnits = []
rnnDO = []
mergingDenseUnits = [1024,512,256]
mergingDO = [0.33,0.33,0.33]
input_dims = []

In [12]:
for i in range(0,4):
  convFilters.append(newConvFilters)
  convKernels.append(newConvKernels)
  convPool.append(newConvPool)
  convDO.append(0.33)
  denseUnits.append([256,128,128])
  denseDO.append([0.33])

convFilters.append([])
convKernels.append([])
convPool.append([])
convDO.append(0.33)
denseUnits.append([512,256,64,64])
denseDO.append([0.33])

convFilters.append(newConvFilters)
convKernels.append(newConvKernels)
convPool.append(newConvPool)
convDO.append(0.33)
denseUnits.append([256,128,128])
denseDO.append([0.33])

for i in range(0,6):
    rnnUnits.append([])
    rnnDO.append(0.2)
    input_dims.append((samplesPerWin, 1, len(channels2Use[i]) ))

# **Make Model**

In [13]:
def shallow_model(input_dim,useBatchNorm,convFilters,convKernels,convPool,convDO,denseUnits,denseDO,rnnUnits,rnnDO,convModelDim):
    
    reg = l2(0.00001)
    model=Sequential()
    model.add(InputLayer(input_shape=( input_dim[0] , input_dim[1], input_dim[2]  ) ))
    if convModelDim == 1:
        model.add(Reshape((input_dim[0], input_dim[2]  ) ))

      
    #Convolutional layers block:
    for i in range(len(convFilters)):
        if convModelDim == 1:
            model.add(Convolution1D(filters=convFilters[i], kernel_size=convKernels[i],
                                            padding='same',kernel_regularizer=reg,bias_regularizer=reg))
        else:
            model.add(Convolution2D(filters=convFilters[i], kernel_size=convKernels[i],
                                            padding='same',kernel_regularizer=reg,bias_regularizer=reg))
        if convPool[i]:
            if convModelDim == 1:
                model.add(MaxPooling1D())
            else:
                model.add(MaxPooling2D())
        if useBatchNorm:
            model.add(BatchNormalization(scale=False))
            model.add(Activation('relu'))
        else:
            model.add(PReLU(shared_axes=[1]))
        model.add(Dropout(convDO))

    ret = True
    for i in range(len(rnnUnits)):
        if i == (len(rnnUnits) -1):
            ret = False            
        model.add(Bidirectional(LSTM(units=rnnUnits[i],return_sequences=ret,
                                             dropout=rnnDO,kernel_regularizer=reg, recurrent_regularizer=reg  )))

    
    #flattening up filters
    if ret:
        model.add(Flatten())

    #
    for i in range(len(denseUnits)):
        model.add(Dense(units=denseUnits[i],kernel_regularizer=reg,bias_regularizer=reg))
        if useBatchNorm:
            model.add(BatchNormalization(scale=False))
            model.add(Activation('relu'))
        else:
            model.add(PReLU(shared_axes=[1]))
        model.add(Dropout(denseDO[min(i,len(denseDO) - 1)]))
  
    return model 
    
def merge_conv_models(useBatchNorm,input_dims,convModelDims,siameseNet,prelu,convFilters,convKernels,convPool,convDO,
                      denseUnits,denseDO,rnnUnits,rnnDO,mergingDenseUnits,mergingDO):
    model_inputs = []
    model_fun = []
    curr_models = []
    for i in range(len(input_dims)):
        curr_models.append(shallow_model(input_dims[i],useBatchNorm,convFilters[i],convKernels[i],convPool[i],
                                             convDO[i],denseUnits[i],denseDO[i],rnnUnits[i],rnnDO[i],convModelDims[i]))
        model_inputs.append(Input(input_dims[i]))
        model_fun.append(curr_models[i](model_inputs[i]))
        
    for _ in range(siameseNet):        
        for i in range(len(input_dims)):
            model_inputs.append(Input(input_dims[i]))
            model_fun.append(curr_models[i](model_inputs[-1]))
    if len(model_fun) > 1:
        merged = concatenate(model_fun)
        
    else:
        merged = model_fun[0]
        
        
    for i in range(len(mergingDenseUnits)):
        merged = Dense(units=mergingDenseUnits[i],kernel_regularizer=l2(0.00001),bias_regularizer=l2(0.00001))(merged)
        merged = PReLU(shared_axes=[1])(merged)
        merged = Dropout(mergingDO[i])(merged)
    pred_layer = Dense(units=2,kernel_regularizer=l2(0.00001),bias_regularizer=l2(0.00001))(merged)
    pred_layer = Activation('softmax')(pred_layer)
    model = Model(model_inputs,pred_layer)
    return model

In [14]:
model = merge_conv_models(useBatchNorm,input_dims,convModelDims,siameseNet,prelu,
                          convFilters,convKernels,convPool,convDO,denseUnits,denseDO,rnnUnits,
                          rnnDO,
                          mergingDenseUnits,
                          mergingDO)    


print(model.summary())

optimizer = Adam(lr=0.00005)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 1400, 1, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 input_4 (InputLayer)           [(None, 1400, 1, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 input_6 (InputLayer)           [(None, 1400, 1, 1)  0           []                               
                                ]                                                             

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


# **Data Generator**

In [15]:
def generator(data_path,batch_size,user_ids,chs,n_sub_sampled,convModelDims,siameseNet, y_alls_pos, y_alls_neg,winSize):
    '''
    This is the actual data generator. Super inefficient as it is...
    '''
    
    flattened_channels = [item for sublist in chs for item in sublist]
    while True:
        x_s = np.zeros((batch_size,n_sub_sampled,len(flattened_channels)),dtype = np.float32)
        if siameseNet >= 1:
            x_s_prev = np.zeros((batch_size,n_sub_sampled,len(flattened_channels)),dtype = np.float32)
            if siameseNet >= 2:
                x_s_next = np.zeros((batch_size,n_sub_sampled,len(flattened_channels)),dtype = np.float32)
            else:
                x_s_next = []
        else:
            x_s_prev = []
            x_s_next = []
        y_s = np.zeros((batch_size),dtype = np.int16)

        #the first half of the batch is taken from positive arousal sample
        for i in range(batch_size//2):
            temp_vec = []
            while len(temp_vec) == 0:
                u_idx = np.random.choice(len(user_ids))
                id = user_ids[u_idx]
                temp_vec = y_alls_pos[u_idx]
            rand_idx = np.random.choice(temp_vec)
            y_s[i] = np.int16(1)
            x_temp = np.asarray(pickle.load(open(data_path + id + '/' + 'x_' + str(rand_idx) + '.p', 'rb')),dtype = np.float32)
            x_temp = x_temp[:,flattened_channels]
            x_s[i] = pre_processing(x_temp,flattened_channels,n_sub_sampled,winSize)
            if siameseNet >= 1:
                prev_file_name = get_prev_file_name(data_path + id +  '/',rand_idx )
                x_temp = np.asarray(pickle.load(open(prev_file_name,'rb')),dtype = np.float32)
                x_temp = x_temp[:,flattened_channels]
                x_s_prev[i]  = pre_processing(x_temp,flattened_channels,n_sub_sampled,winSize)                    
                if siameseNet == 2:
                    len_yall = len(y_alls_pos[u_idx]) + len(y_alls_neg[u_idx])
                    next_file_name = get_next_file_name(data_path + id +  '/',rand_idx,len_yall)
                    x_temp = np.asarray(pickle.load(open(next_file_name,'rb')),dtype = np.float32)
                    x_temp = x_temp[:,flattened_channels]
                    x_s_next[i] = pre_processing(x_temp,flattened_channels,n_sub_sampled,winSize)
        
        #the second half of the batch is taken from negative arousal sample          
        for i in range(batch_size//2,batch_size):
            u_idx = np.random.choice(len(user_ids))
            id = user_ids[u_idx]
            rand_idx = np.random.choice(y_alls_neg[u_idx])
            y_s[i] = np.int16(0)
            x_temp = np.asarray(pickle.load(open(data_path + id + '/' + 'x_' + str(rand_idx) + '.p', 'rb')),dtype = np.float32)
            x_temp = x_temp[:,flattened_channels]
            x_s[i] = pre_processing(x_temp,flattened_channels,n_sub_sampled,winSize)
            if siameseNet >= 1:
                prev_file_name = get_prev_file_name(data_path + id +  '/',rand_idx )
                x_temp = np.asarray(pickle.load(open(prev_file_name,'rb')),dtype = np.float32)
                x_temp = x_temp[:,flattened_channels]
                x_s_prev[i]  = pre_processing(x_temp,flattened_channels,n_sub_sampled,winSize)                    
                if siameseNet == 2:
                    len_yall = len(y_alls_pos[u_idx]) + len(y_alls_neg[u_idx])
                    next_file_name = get_next_file_name(data_path + id +  '/',rand_idx,len_yall)
                    x_temp = np.asarray(pickle.load(open(next_file_name,'rb')),dtype = np.float32)
                    x_temp = x_temp[:,flattened_channels]
                    x_s_next[i]  = pre_processing(x_temp,flattened_channels,n_sub_sampled,winSize)
   
                    
        x_s = rearrange_channels(x_s,convModelDims,chs)           
        
        if siameseNet >= 1:
            x_s_prev = rearrange_channels(x_s_prev,convModelDims,chs)   
            if siameseNet == 2:
                x_s_next = rearrange_channels(x_s_next,convModelDims,chs) 
        
        yield x_s + x_s_prev + x_s_next, to_categorical(y_s,num_classes = 2)
    
   
def comp_inst_phase(x_temp):
    fs = 50.0    #carefull! sampling frequency is hard coded here...
    analytic_signal = hilbert(x_temp)
    instantaneous_phase = np.unwrap(np.angle(analytic_signal))
    instantaneous_frequency = (np.diff(instantaneous_phase) /(2.0*np.pi) * fs)
    instantaneous_frequency = np.append(instantaneous_frequency,instantaneous_frequency[-1])
    return instantaneous_frequency
    
def per_sample_x_axis_sub_sample(x_train,x_axis_size):
    '''
    cropping time signlas for data augmentation
    '''
    diff = x_train.shape[0] - x_axis_size 
    intIdx = np.random.randint(diff)
    finIdx = diff - intIdx
    return x_train[intIdx:(x_train.shape[0]-finIdx),:]

def pre_processing(x_temp,flattened_channels,n_sub_sampled,winSize, normFlag = True):
    #pre-processing of signals. This include standardisation, and frequency computation for EEG
    data_means = [-1.58837489e-04, -1.11909714e-04,  1.85214086e-05,  3.43906861e-06,
                      -4.20280479e-04, -4.93160169e-06,  1.79198553e-05,  7.16557753e-05,
                      -4.05202774e-03, -1.71058490e-01, -1.55950109e-02,  9.34850143e+01,
                      -1.74383175e-06]
    data_std = [3.82136545e+01, 4.03138769e+01, 3.92392071e+01, 4.47130111e+01,
                    3.96919219e+01, 4.08547734e+01, 3.77899386e+01, 5.63194308e+01,
                    6.32337194e+02, 3.99307484e+02, 5.43303006e+02, 8.40357161e+00,
                    2.09212648e-01]
    if normFlag:
        for j in range(x_temp.shape[1]):
            x_temp[:,j] = (x_temp[:,j] - data_means[flattened_channels[j]])/data_std[flattened_channels[j]]
    for j in range(len(flattened_channels)):
        if (flattened_channels[j]>=0) and (flattened_channels[j]<=5): # if it is EEG channel
            x_temp[:,j] = comp_inst_phase(x_temp[:,j])
    if n_sub_sampled < x_temp.shape[0]:
        x_s_i = per_sample_x_axis_sub_sample(x_temp,n_sub_sampled)
    else:
        x_s_i = x_temp
    return x_s_i
  
def get_new_indexing(channels2Use):
    channels2Use_flat = [item for sublist in channels2Use for item in sublist]
    new_indexing = []
    for ch_list in channels2Use: 
        curr_new_indexing = []
        for ch in ch_list:
            curr_new_indexing.append([i for i, c in enumerate(channels2Use_flat) if c == ch][0])
        new_indexing.append(curr_new_indexing)
    
    return new_indexing

def rearrange_channels(x_s,convModelDims,chs):
    '''
    re-arranging channels in a way that is compatible with keras input for training
    '''
    x_s = x_s[:,:,:,np.newaxis]
    x_s = [x_s[:,:,c_s,:] if convModelDims[j] == 2 else np.transpose(x_s[:,:,c_s,:],[0,1,3,2])
            for j, c_s in enumerate(get_new_indexing(chs))]
    return x_s

def get_prev_file_name(preName,rand_idx):
    '''
    utils to get name of previous data sample. This is an ugly function really, it is necessary because of bad choices made at the
    beginning of the coding. It should be removed, and things should get cleaned up a bit
    '''
    if os.path.isfile(preName + 'x_prev_' + str(rand_idx) + '.p'):
        prev_file_name = preName + 'x_prev_' + str(rand_idx) + '.p'
    else:
        prevIdx = max(rand_idx - 1,0)
        prev_file_name = preName + 'x_' + str(prevIdx) + '.p'
    return prev_file_name

def get_next_file_name(preName,rand_idx,len_y_s):
    '''
    same as above but for the successive input sample
    '''
    if os.path.isfile(preName + 'x_next_' + str(rand_idx) + '.p'):
        next_file_name = preName + 'x_next_' + str(rand_idx) + '.p'
    else:
        nextIdx = min(rand_idx + 1,len_y_s - 1)
        next_file_name = preName + 'x_' + str(nextIdx) + '.p'
    return next_file_name


# **Scoring**

In [16]:
import argparse

class Challenge2018Score:
    """Class used to compute scores for the 2018 PhysioNet/CinC Challenge.
    A Challenge2018Score object aggregates the outputs of a proposed
    classification algorithm, and calculates the area under the
    precision-recall curve, as well as the area under the receiver
    operating characteristic curve.
    After creating an instance of this class, call score_record() for
    each record being tested.  To calculate scores for a particular
    record, call record_auprc() and record_auroc().  After scoring all
    records, call gross_auprc() and gross_auroc() to obtain the scores
    for the database as a whole.
    """

    def __init__(self, input_digits=None):
        """Initialize a new scoring buffer.
        If 'input_digits' is given, it is the number of decimal digits
        of precision used in input probability values.
        """
        if input_digits is None:
            input_digits = 3
        self._scale = 10**input_digits
        self._pos_values = np.zeros(self._scale + 1, dtype=np.int64)
        self._neg_values = np.zeros(self._scale + 1, dtype=np.int64)
        self._record_auc = {}

    def score_record(self, truth, predictions, record_name=None):
        """Add results for a given record to the buffer.
        'truth' is a vector of arousal values: zero for non-arousal
        regions, positive for target arousal regions, and negative for
        unscored regions.
        'predictions' is a vector of probabilities produced by the
        classification algorithm being tested.  This vector must be
        the same length as 'truth', and each value must be between 0
        and 1.
        If 'record_name' is specified, it can be used to obtain
        per-record scores afterwards, by calling record_auroc() and
        record_auprc().
        """
        # Check if length is correct
        if len(predictions) != len(truth):
            raise ValueError("length of 'predictions' does not match 'truth'")

        # Compute the histogram of all input probabilities
        b = self._scale + 1
        r = (-0.5 / self._scale, 1.0 + 0.5 / self._scale)
        all_values = np.histogram(predictions, bins=b, range=r)[0]

        # Check if input contains any out-of-bounds or NaN values
        # (which are ignored by numpy.histogram)
        if np.sum(all_values) != len(predictions):
            raise ValueError("invalid values in 'predictions'")

        # Compute the histogram of probabilities within arousal regions
        pred_pos = []
        pred_ign = []
        i = 0
        for x in truth:
          if int(x) > int(0):
            pred_pos.append(predictions[i])
          if int(x) < 0:
            pred_ign.append(predictions[i])   
          i += 1      
        pos_values = np.histogram(pred_pos, bins=b, range=r)[0]

        # Compute the histogram of probabilities within unscored regions
        ign_values = np.histogram(pred_ign, bins=b, range=r)[0]

        # Compute the histogram of probabilities in non-arousal regions,
        # given the above
        neg_values = all_values - pos_values - ign_values

        self._pos_values += pos_values
        self._neg_values += neg_values

        if record_name is not None:
            self._record_auc[record_name] = self._auc(pos_values, neg_values)

    def _auc(self, pos_values, neg_values):
        # Calculate areas under the ROC and PR curves by iterating
        # over the possible threshold values.

        # At the minimum threshold value, all samples are classified as
        # positive, and thus TPR = 1 and TNR = 0.
        tp = np.sum(pos_values)
        fp = np.sum(neg_values)
        tn = fn = 0
        tpr = 1
        tnr = 0
        if tp == 0 or fp == 0:
            # If either class is empty, scores are undefined.
            return (float('nan'), float('nan'))
        ppv = float(tp) / (tp + fp)
        auroc = 0
        auprc = 0

        # As the threshold increases, TP decreases (and FN increases)
        # by pos_values[i], while TN increases (and FP decreases) by
        # neg_values[i].
        for (n_pos, n_neg) in zip(pos_values, neg_values):
            tp -= n_pos
            fn += n_pos
            fp -= n_neg
            tn += n_neg
            tpr_prev = tpr
            tnr_prev = tnr
            ppv_prev = ppv
            tpr = float(tp) / (tp + fn)
            tnr = float(tn) / (tn + fp)
            if tp + fp > 0:
                ppv = float(tp) / (tp + fp)
            else:
                ppv = ppv_prev
            auroc += (tpr_prev - tpr) * (tnr + tnr_prev) * 0.5
            auprc += (tpr_prev - tpr) * ppv_prev
        return (auroc, auprc)

    def gross_auroc(self):
        """Compute the area under the ROC curve.
        The result will be NaN if none of the records processed so far
        contained any target arousals.
        """
        return self._auc(self._pos_values, self._neg_values)[0]

    def gross_auprc(self):
        """Compute the area under the precision-recall curve.
        The result will be NaN if none of the records processed so far
        contained any target arousals.
        """
        return self._auc(self._pos_values, self._neg_values)[1]

    def record_auroc(self, record_name):
        """Compute the area under the ROC curve for a single record.
        The result will be NaN if the record did not contain any
        target arousals.
        The given record must have previously been processed by
        calling score_record().
        """
        return self._record_auc[record_name][0]

    def record_auprc(self, record_name):
        """Compute the area under the PR curve for a single record.
        The result will be NaN if the record did not contain any
        target arousals.
        The given record must have previously been processed by
        calling score_record().
        """
        return self._record_auc[record_name][1]


################################################################
# Command line interface
################################################################


# p = argparse.ArgumentParser()
# p.add_argument('vecfiles', metavar='RECORD.vec', nargs='+',
#                 help='vector of probabilities to score')
# p.add_argument('-r', '--reference-dir', metavar='DIR', default='training',
#                 help='location of reference arousal.mat files')
# args = p.parse_args()

# print('Record          AUROC     AUPRC')
# print('_______________________________')
# s = Challenge2018Score()
# failed = 0
# for vec_file in args.vecfiles:
#     record = os.path.basename(vec_file)
#     if record.endswith('.vec'):
#         record = record[:-4]

#     arousal_file = os.path.join(args.reference_dir, record,
#                                 record + '-arousal.mat')
#     try:
#         # Load reference annotations from the arousal.mat file
#         with h5py.File(arousal_file, 'r') as af:
#             truth = np.ravel(af['data']['arousals'])

#         # Load predictions from the vec file
#         predictions = np.zeros(len(truth), dtype=np.float32)
#         with open(vec_file, 'rb') as vf:
#             i = -1
#             for (i, v) in enumerate(vf):
#                 try:
#                     predictions[i] = v
#                 except IndexError:
#                     break
#             if i != len(truth) - 1:
#                 print('Warning: wrong number of samples in %s'
#                       % vec_file)

#         # Compute and print scores for this record
#         s.score_record(truth, predictions, record)
#         auroc = s.record_auroc(record)
#         auprc = s.record_auprc(record)
#         print('%-11s  %8.6f  %8.6f' % (record, auroc, auprc))
#     except Exception as exc:
#         print(exc)
#         print('%-11s  %8s  %8s' % (record, 'error', 'error'))
#         failed = 1

# # Compute and print overall scores
# auroc = s.gross_auroc()
# auprc = s.gross_auprc()
# print('_______________________________')
# print('%-11s  %8.6f  %8.6f' % ('Overall', auroc, auprc))
# # sys.exit(failed)

# **Model Fitting**

In [17]:
class ModelTest(Callback):
    '''
    Definition of callback used for computation of AUC on the valdiation set. Either provide Xt and y_dense (that is the numpy arrays of data)
    or the path to data directory and a list of user ids.
    '''
    def __init__(self, test_every_X_epochs, batch_size, rep_idx, valUserIDs,
                 dataDir, samplesPerWin, channels2Use, convModelDims, siameseNet, modelID, useGenerator,winSize ):
        super(ModelTest, self).__init__()
        
        self.test_every_X_epochs = test_every_X_epochs
        self.batch_size = batch_size
        self.bestAUPR = 0.
        self.AUPRhist = []
        self.rep_idx = rep_idx
        self.valUserIDs = valUserIDs
        self.dataDir = dataDir
        self.samplesPerWin = samplesPerWin
        self.channels2Use = channels2Use
        self.convModelDims = convModelDims
        self.siameseNet = siameseNet
        self.modelID = modelID
        self.y = self.init_y_field()
        self.useGenerator = useGenerator
        self.winSize = winSize
        if not self.useGenerator:
            self.cache_Xt()
            self.Xt = []
            
    #initilise labels for the validation set            
    def init_y_field(self):
        y = []
        for currUser in self.valUserIDs:
            y.append(np.int16( pickle.load(open(self.dataDir + currUser + '/' + 'y.p', 'rb'))))
        return y
    
    
    # Here, I build up the input data matrix for each user, and I cache it to file for quick RAM menagement.
    # It's an ugly solution, it would probably much better to just define some proper data generator for the validation test as well...
    def cache_Xt(self):

        Xt = []
        
        flattened_channels = [item for sublist in self.channels2Use for item in sublist]
        start_time = time.time()
        
        for i,currUser in enumerate(self.valUserIDs):
            sys.stdout.write("\r" + 'pre-processing for validation user #: ' +  str(i))
            sys.stdout.flush()
            x_s = np.zeros((len(self.y[i]),self.samplesPerWin,len(flattened_channels)),dtype = np.float32)
            if self.siameseNet >= 1:
                x_s_prev = np.zeros((len(self.y[i]),self.samplesPerWin,len(flattened_channels)),dtype = np.float32)
                if self.siameseNet >= 2:
                    x_s_next = np.zeros((len(self.y[i]),self.samplesPerWin,len(flattened_channels)),dtype = np.float32)
                else:
                    x_s_next = []
            else:
                x_s_prev = []
                x_s_next = []
            
            for j in range(len(self.y[i])):
                x_temp = np.asarray(pickle.load(open(self.dataDir + currUser + '/' + 'x_' + str(j) + '.p','rb')),dtype = np.float32)
                x_temp = x_temp[:,flattened_channels]
                x_s[j] = pre_processing(x_temp,flattened_channels,self.samplesPerWin,self.winSize)
                if self.siameseNet >= 1:
                    prev_file_name = get_prev_file_name(self.dataDir + currUser +  '/',j )
                    x_temp = np.asarray(pickle.load(open(prev_file_name, 'rb')),dtype = np.float32)
                    x_temp = x_temp[:,flattened_channels]
                    x_s_prev[j]  = pre_processing(x_temp,flattened_channels,self.samplesPerWin,self.winSize)
                    if self.siameseNet == 2:
                        next_file_name = get_next_file_name(self.dataDir  + currUser +  '/',j,len(self.y[i]))
                        x_temp = np.asarray(pickle.load(open(next_file_name, 'rb')),dtype = np.float32)
                        x_temp = x_temp[:,flattened_channels]
                        x_s_next[j]  = pre_processing(x_temp,flattened_channels,self.samplesPerWin,self.winSize)
            x_s = rearrange_channels(x_s,self.convModelDims,self.channels2Use)
            if self.siameseNet >= 1:
                x_s_prev = rearrange_channels(x_s_prev,self.convModelDims,self.channels2Use)   
                if self.siameseNet == 2:
                    x_s_next = rearrange_channels(x_s_next,self.convModelDims,self.channels2Use) 
            Xt.append(x_s + x_s_prev + x_s_next)
        print() 
        print('elapsed time for validation set pre-processing: ' + str(time.time() - start_time))
        with open(self.dataDir + 'cached_Xt.p','wb') as f:
            pickle.dump(Xt,f)
        return 
    
    # Simple method that just loads the model that was pickled in the function above.
    def load_Xt(self):
        with open(self.dataDir + 'cached_Xt.p', 'rb') as f:
            Xt = pickle.load(f)
        return Xt
    
    #method called by keras at the beginning of each epoch. In use it to compute statistics on validation and test set
    def on_epoch_begin(self, epoch, logs={}):
        if epoch % self.test_every_X_epochs != 0:
            return
        start_time = time.time()
        
        
        scoreObj = Challenge2018Score() #defining scoring object
        self.Xt = self.load_Xt() #loading cached validation set
        
        #looping on all the validation users
        for i, currUser in enumerate(self.valUserIDs):
            prediction = self.model.predict(self.Xt[i], batch_size=self.batch_size)
            if len(self.y[i]) > len(prediction):
              self.y[i] = self.y[i][:len(prediction)]
            prediction = prediction[:len(self.y[i]),1]
            scoreObj.score_record(self.y[i], prediction, currUser)
            
        self.Xt = []    #emptying the Xt field
        auroc = scoreObj.gross_auroc()
        auprc = scoreObj.gross_auprc()
        self.AUPRhist.append(auprc)
        
        if auprc > self.bestAUPR:
            self.bestAUPR = auprc        
            model_json = self.model.to_json()
            fileName = self.dataDir + self.modelID +  "_model"
            with open(fileName + ".json", "w+") as json_file:
                json_file.write(model_json)
            self.model.save_weights(fileName + ".h5")
            print('---------- Saving model to file, stats are: ----------')

        AUCStr = ("%0.4f" % auroc )
        AUPRStr = ("%0.4f" % auprc )
        print("Win_AUC on validation set " + AUCStr + " AUPR: " + AUPRStr)  
        
        print("Elapsed time: %s s" % (time.time() - start_time))

In [ ]:
########################################################
###DEFINING CALLBACKS AND STARTING MODEL FITTING...#####
########################################################

if training2do:
    valSetCallBack = ModelTest(1, batch_size, np.nan, valUserIDs,
                 data_path, samplesPerWin, channels2Use, convModelDims, siameseNet, modelID,useGenerator,winSize)
    testSetCallBack = ModelTest(1, batch_size, np.nan, testUserIDs,
                 data_path, samplesPerWin, channels2Use, convModelDims, siameseNet, modelID,useGenerator,winSize)
    callbacks_list = [valSetCallBack,testSetCallBack]
    
    y_alls = []
    y_alls_pos = []
    y_alls_neg = []
    for currUser in userIDs:
        #y_alls.append(np.int16( pickle.load(open(data_path+ currUser + '/' + 'y.p'))))
        aux = np.int16( pickle.load(open(data_path+ currUser + '/' + 'y.p', 'rb')))
        y_alls_pos.append(np.nonzero(aux == 1)[0])
        y_alls_neg.append(np.nonzero(aux == 0)[0])
        
  
    #model fitting on data generators
    hist = model.fit_generator(generator(data_path,batch_size,userIDs,channels2Use,samplesPerWin,convModelDims,siameseNet,y_alls_pos ,y_alls_neg,winSize),
                               steps_per_epoch = (n_samples/batch_size), epochs = n_ep, use_multiprocessing = True,
                               callbacks = callbacks_list, workers = workers, max_queue_size = 10)

pre-processing for validation user #: 0
elapsed time for validation set pre-processing: 0.5692996978759766
pre-processing for validation user #: 0
elapsed time for validation set pre-processing: 0.824427604675293


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


---------- Saving model to file, stats are: ----------
Win_AUC on validation set 0.4967 AUPR: 0.1818
Elapsed time: 15.122232675552368 s
---------- Saving model to file, stats are: ----------
Win_AUC on validation set 0.5157 AUPR: 0.0902
Elapsed time: 12.085102319717407 s
Epoch 1/30
102/102 [==============================] - 698s 7s/step - loss: 0.8719 - accuracy: 0.5091
Win_AUC on validation set 0.4358 AUPR: 0.1359
Elapsed time: 10.991071701049805 s
Win_AUC on validation set 0.4988 AUPR: 0.0823
Elapsed time: 11.004363059997559 s
Epoch 2/30
102/102 [==============================] - 611s 6s/step - loss: 0.7936 - accuracy: 0.5255
Win_AUC on validation set 0.4983 AUPR: 0.1710
Elapsed time: 11.104629755020142 s
---------- Saving model to file, stats are: ----------
Win_AUC on validation set 0.5903 AUPR: 0.1205
Elapsed time: 11.282002449035645 s
Epoch 3/30
102/102 [==============================] - 610s 6s/step - loss: 0.7760 - accuracy: 0.5522
Win_AUC on validation set 0.4489 AUPR: 0.1451


# **Validating and Testing**

In [ ]:
def get_data_windows(currMatFilePath,win_size_fs,downSampling,overlap,test):
    val, _ = wfdb.rdsamp(currMatFilePath)
    val = np.float32(val)
    curr_user_n_dense_samples = val.shape[0]
    val = val[::downSampling,:]
    if not test:
        with h5py.File(currMatFilePath+'-arousal.mat', 'r') as f:
            arousalLabel = f['data']['arousals']
            arousalLabel = arousalLabel[:]
        arousalLabel = arousalLabel[::downSampling]
    length = len(val)
    startIdx = 0
    finIdx = win_size_fs
    deltaWindow = int(np.floor((1 - overlap)*win_size_fs))

    winCount = 1
    x_temp = []
    y_temp = []
    while finIdx <= length:
        winVal = val[startIdx:finIdx,:]
        x_temp.append(winVal)
        currLabelVec = np.array(arousalLabel[startIdx:finIdx]);
        currLabelVoting = [np.sum(currLabelVec == -1),np.sum(currLabelVec == 0),
                           np.sum(currLabelVec == 1)]
        idx = np.argmax(currLabelVoting);
        idx = idx - 1;
        
        y_temp.append(    idx    )
        winCount = winCount + 1
        startIdx = startIdx + deltaWindow
        finIdx = finIdx + deltaWindow
        
    x_temp = np.array(x_temp)
    return x_temp, curr_user_n_dense_samples, y_temp



def from_win_prediction_to_dense(model_output,win_upsample,overlap,len_y_dense):
    '''
        Uses the prediction of overlapping time windows (model_output) to compute dense prediction (that is @200Hz)
    '''
    dense_pred = np.zeros((len_y_dense,),dtype = np.float16)
    count_pred = np.zeros((len_y_dense,),dtype = np.int16)
    delta = int(np.floor((1.0-overlap)*win_upsample))
    init = 0
    fin = int(win_upsample)
    for i in range(model_output.shape[0]):
        dense_pred[init:fin] = dense_pred[init:fin] + model_output[i]
        count_pred[init:fin] = count_pred[init:fin] + 1 
        init = init + delta
        fin = fin + delta
    
    for i ,count in enumerate(count_pred):
        if count == 0:
            count_pred[i] = 1
            
        
    dense_pred = dense_pred / count_pred
    

    return dense_pred      




def prepare_input_data(x_temp,samplesPerWin,siameseNet,channels2Use,convModelDims,overlap,winSize):
    flattened_channels = [item for sublist in channels2Use for item in sublist]
    x_temp = x_temp[:,:,flattened_channels]
    x_s = np.zeros((len(x_temp),samplesPerWin,len(flattened_channels)),dtype = np.float32)
    if siameseNet >= 1:
        x_s_prev = np.zeros((len(x_temp),samplesPerWin,len(flattened_channels)),dtype = np.float32)
        if siameseNet >= 2:
            x_s_next = np.zeros((len(x_temp),samplesPerWin,len(flattened_channels)),dtype = np.float32)
        else:
            x_s_next = []
    else:
        x_s_prev = []
        x_s_next = []

    for i in range(len(x_temp)):
        x_s[i] = pre_processing(x_temp[i],flattened_channels,samplesPerWin,winSize)
    
    if siameseNet >= 1:
        for i in range(len(x_temp)):
            x_s_prev[i]  = pre_processing(x_temp[max(0,i-int(1./(1-overlap) ) )  ],flattened_channels,samplesPerWin,winSize, normFlag = False)
    if siameseNet == 2:
        for i in range(len(x_temp)):
            x_s_next[i]  = pre_processing(x_temp[min(i+int(1./(1-overlap)),len(x_temp) - 1)],flattened_channels,samplesPerWin,winSize, normFlag = False)
        
    x_s = rearrange_channels(x_s,convModelDims,channels2Use)           
    if siameseNet >= 1:
        x_s_prev = rearrange_channels(x_s_prev,convModelDims,channels2Use)   
    if siameseNet == 2:
        x_s_next = rearrange_channels(x_s_next,convModelDims,channels2Use) 
    return x_s + x_s_prev + x_s_next

def get_raw_data_path():
    raw_data_path = '../data/training/'
    return raw_data_path

In [ ]:
####################################################
#Trying out the model on dense prediction settings#
###################################################

from submission_utils import get_data_windows, from_win_prediction_to_dense, prepare_input_data, get_raw_data_path
from new_scoring_function import Challenge2018Score
import h5py


downSampling = 4
overlap = 0.5
repData = 5


modelPath = data_path + modelID +  '_model'
raw_data_path = get_raw_data_path() 

json_file = open(modelPath + '.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights(modelPath + '.h5')
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

scoreObj = Challenge2018Score()
window_scoreObj = Challenge2018Score()





if testing2do:
    for hhh, curr_user in enumerate(test_users):
        sys.stdout.write("\r" + 'iteration: ' + str(hhh) + ' user: ' + curr_user)
        sys.stdout.flush()
        #print 'iteration: ' + str(hhh) + ' user: ' + curr_user
        currMatFilePath = test_data_dir + curr_user + '/' + curr_user
        y_window = [];
        for _ in range(repData):
            x_temp, curr_user_n_dense_samples, _  = get_data_windows(currMatFilePath,int(round(winSize*(200.0/float(downSampling)))),downSampling,overlap,True)
            x_temp = prepare_input_data(x_temp,samplesPerWin,siameseNet,channels2Use,convModelDims,overlap,winSize)
            if not len(y_window):
                y_window = model.predict(x_temp)
                y_window = y_window[:,1]
            else:
                temp_win = model.predict(x_temp)
                y_window = y_window + temp_win[:,1]
        y_window = y_window/repData
        y_pred = np.round(from_win_prediction_to_dense(y_window, winSize*200,overlap,curr_user_n_dense_samples),3)
        y_pred = y_pred[:,np.newaxis]
        np.savetxt("../annotations/" + curr_user + ".vec", y_pred, fmt='%.3f')




if validation2do:
    
    scoreObj = Challenge2018Score()
    window_scoreObj = Challenge2018Score()
    
    for hhh, curr_user in enumerate(valUserIDs):
        sys.stdout.write("\r" + 'iteration: ' + str(hhh) + ' user: ' + curr_user)
        sys.stdout.flush()
        currMatFilePath = raw_data_path + curr_user + '/' + curr_user
        y_window = [];
        for _ in range(repData):
            x_temp, curr_user_n_dense_samples, y_temp = get_data_windows(currMatFilePath,int(round(winSize*(200.0/float(downSampling)))),downSampling,overlap,False)
            x_temp = prepare_input_data(x_temp,samplesPerWin,siameseNet,channels2Use,convModelDims,overlap,winSize)
            if not len(y_window):
                y_window = model.predict(x_temp)
                y_window = y_window[:,1]
            else:
                temp_win = model.predict(x_temp)
                y_window = y_window + temp_win[:,1]
        y_window = y_window/repData
                
            
    
    
        
        
        y_pred = np.round(from_win_prediction_to_dense(y_window, winSize*200,overlap,curr_user_n_dense_samples),3)
        y_pred = y_pred[:,np.newaxis]
        y_window = y_window[:,np.newaxis]
        arousalFile = currMatFilePath + '-arousal.mat'
        with h5py.File(arousalFile, 'r') as f:
            arousalLabel = f['data']['arousals']
            arousalLabel = arousalLabel[:]
        window_scoreObj.score_record(np.asarray(y_temp)[np.asarray(y_temp) >= 0 ],y_window[np.asarray(y_temp) >= 0 ],curr_user)
        scoreObj.score_record(arousalLabel, y_pred, curr_user)
    print 
    print 'Dense level labelling: '
    auroc = scoreObj.gross_auroc()
    auprc = scoreObj.gross_auprc()
    print 'AUROC: ' + str(auroc) 
    print 'AUPR: ' + str(auprc) 
    
    print 'Window level labelling: '                 
    auroc = window_scoreObj.gross_auroc()
    auprc = window_scoreObj.gross_auprc()
    print 'AUROC: ' + str(auroc) 
    print 'AUPR: ' + str(auprc) 